# 🎂 Train Custom Cake AI Model + Export to ONNX

## Setup:
1. **Runtime → Change runtime type → T4 GPU**
2. Run all cells in order

---

In [ ]:
# Step 1: Install Required Libraries
!pip install -q diffusers==0.25.0 transformers accelerate peft datasets huggingface_hub
!pip install -q onnx onnxruntime-gpu optimum[onnxruntime]
!pip install -q bitsandbytes safetensors
print("✅ Libraries installed!")

In [ ]:
# Step 2: Login to Hugging Face
from huggingface_hub import notebook_login
print("🔐 Get token from: https://huggingface.co/settings/tokens\n")
notebook_login()

In [ ]:
# Step 3: Load Your Cake Images Dataset
from datasets import load_dataset
import matplotlib.pyplot as plt

print("📥 Loading cake images dataset...")
dataset = load_dataset("praveen9025/cake-images", split="train")
print(f"✅ Loaded {len(dataset)} images!")

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for i, ax in enumerate(axes):
    ax.imshow(dataset[i]['image'])
    ax.axis('off')
plt.show()

In [ ]:
# Step 4: Prepare Training Data
import os
from PIL import Image

TRAIN_DIR = "cake_training_data"
os.makedirs(TRAIN_DIR, exist_ok=True)

CAPTION = "a professional photo of cakestyle cake, bakery, studio lighting"

print("📁 Preparing training data...")
for i, item in enumerate(dataset):
    img = item['image'].convert('RGB').resize((512, 512), Image.LANCZOS)
    img.save(os.path.join(TRAIN_DIR, f"cake_{i:03d}.png"))
    with open(os.path.join(TRAIN_DIR, f"cake_{i:03d}.txt"), 'w') as f:
        f.write(CAPTION)

print(f"✅ Prepared {len(dataset)} images!")

In [ ]:
# Step 5: Clone Diffusers
!git clone https://github.com/huggingface/diffusers.git
!pip install -q -r diffusers/examples/dreambooth/requirements_sdxl.txt
print("✅ Training scripts ready!")

In [ ]:
# Step 6: Configure Accelerate
!accelerate config default
print("✅ Accelerate configured!")

In [ ]:
# Step 7: Train LoRA Model (30-60 minutes)
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print("🚀 Starting training... ☕ This takes 30-60 minutes\n")

!accelerate launch diffusers/examples/dreambooth/train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --instance_data_dir="cake_training_data" \
  --output_dir="cake-lora-model" \
  --instance_prompt="a photo of cakestyle cake" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --gradient_checkpointing \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=500 \
  --rank=16 \
  --mixed_precision="fp16" \
  --seed=42

print("\n✅ Training complete!")

In [ ]:
# Step 8: Check what files were created
import os

print("📁 Files in cake-lora-model:")
for root, dirs, files in os.walk("cake-lora-model"):
    for file in files:
        filepath = os.path.join(root, file)
        size_mb = os.path.getsize(filepath) / (1024*1024)
        print(f"  {filepath} ({size_mb:.2f} MB)")

In [ ]:
# Step 9: Load and Merge LoRA (FIXED VERSION)
from diffusers import DiffusionPipeline, AutoencoderKL
import torch
import os

print("🔗 Merging LoRA weights into base model...")

# Load base model
vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix", 
    torch_dtype=torch.float16
)

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
).to("cuda")

# Find the correct LoRA weights file
lora_dir = "cake-lora-model"
possible_files = [
    "pytorch_lora_weights.safetensors",
    "pytorch_lora_weights.bin",
]

lora_file = None
for f in possible_files:
    if os.path.exists(os.path.join(lora_dir, f)):
        lora_file = f
        break

if lora_file:
    print(f"📄 Found LoRA weights: {lora_file}")
    pipe.load_lora_weights(lora_dir, weight_name=lora_file)
    pipe.fuse_lora()
    print("✅ LoRA weights merged!")
else:
    print("❌ No LoRA weights found. Check training output.")
    print(f"Files in {lora_dir}:", os.listdir(lora_dir) if os.path.exists(lora_dir) else "Directory not found")

In [ ]:
# Step 10: Test the merged model
import matplotlib.pyplot as plt

print("🎨 Testing merged model...")

prompt = "a photo of cakestyle chocolate birthday cake with sprinkles"
image = pipe(prompt, num_inference_steps=25).images[0]

plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.axis('off')
plt.title("Test: " + prompt[:40] + "...")
plt.savefig("test_result.png")
plt.show()

print("✅ Model works!")

In [ ]:
# Step 11: Save merged model
MERGED_MODEL_PATH = "cake-model-merged"

print(f"💾 Saving merged model to {MERGED_MODEL_PATH}...")
pipe.save_pretrained(MERGED_MODEL_PATH)
print("✅ Merged model saved!")

In [ ]:
# Step 12: Export to ONNX
from optimum.onnxruntime import ORTStableDiffusionXLPipeline

ONNX_OUTPUT_DIR = "cake-model-onnx"

print("📦 Exporting to ONNX format...")
print("⏱️ This takes 10-20 minutes...\n")

onnx_pipe = ORTStableDiffusionXLPipeline.from_pretrained(
    MERGED_MODEL_PATH,
    export=True,
)

onnx_pipe.save_pretrained(ONNX_OUTPUT_DIR)

print(f"\n✅ ONNX model saved to: {ONNX_OUTPUT_DIR}")
!ls -la {ONNX_OUTPUT_DIR}

In [ ]:
# Step 13: Download ONNX Model
import shutil
import os

ZIP_NAME = "cake-model-onnx.zip"

print(f"📦 Creating {ZIP_NAME}...")
shutil.make_archive("cake-model-onnx", 'zip', ONNX_OUTPUT_DIR)

print(f"\n✅ Created: {ZIP_NAME}")
print(f"📁 Size: {os.path.getsize(ZIP_NAME) / (1024*1024*1024):.2f} GB")
print(f"\n⬇️ Download from Files panel on the left")

In [ ]:
# Step 14: Save to Google Drive
from google.colab import drive
import shutil

drive.mount('/content/drive')

# Copy zip to Drive
shutil.copy("cake-model-onnx.zip", "/content/drive/MyDrive/")

print("\n✅ Saved to Google Drive!")
print("📁 Google Drive > cake-model-onnx.zip")

---
## ✅ Done!

Your ONNX model is ready!

**Usage:**
```python
from optimum.onnxruntime import ORTStableDiffusionXLPipeline

pipe = ORTStableDiffusionXLPipeline.from_pretrained("cake-model-onnx")
image = pipe("a photo of cakestyle chocolate cake").images[0]
```

---